# Interpolation
## Polynom Interpolation

Um zwischen Punkten zu interpolieren, kann eine Polynom-Interpolation durchgeführt werden.

Das Ziel einer Polynom-Interpolation ist es, ein Polynom zu finden, welches durch die gegebenen Punkte (Stützstellen) geht.

Ein Polynom ist wie folgt definiert:

$\sum_{i=0}^{n} c_i t^i$

n ist das Grad des Polynoms, das gesuchte Polynom hat ein Grad von m-1, wobei m für die Anzahl der gegebenen Punkte steht.

$c_i$ sind die Koeffizienten des Polynom.

Um ein Polynom $f$ zu finden, welches durch die gegebenen Punkte geht, könnten wir folgendes Gleichungssystem aufstellen:
$c_i$ * $t^i$ = $f(t)$

Eine Stützstelle ist gegeben durch ($t$,$f(t)$), wir suchen also die Koeffizienten des Polynoms.

$$
\left(\begin{array}{cc} 
 1 & t_0 & t_0^2 & ... & t_0^n\\
1 & t_1 & t_1^2 & ... & t_1^n \\
.\\
.\\
1 & t_n & t_n^2 & ... & t_n^n
\end{array}\right)
\left(\begin{array}{cc} 
 c_0\\
c_1\\
.\\
.\\
c_n 
\end{array}\right)
=
\left(\begin{array}{cc} 
 f_0\\
f_1\\
.\\
.\\
f_n 
\end{array}\right)
$$ 

Die entstandene Matrix nennt sich Vandermonde Matrix, um die eindeutige Lösbarkeit eines Gleichungssystems zu bestimmten, können wir prüfen ob $det A \neq 0$  

Die Determinante  einer Vandermonde Matrix lässt sich wie folgt berechnen: $\prod_{i,j=1;j>i}^{n} (t_i - t_j)$ Daraus folgt $det A \neq 0$ wenn $t_i \neq t_j$

Hier ein kleines Porgramm zum Üben von Polynom-Interpolation:
Schreibe dazu die Koeffezienten mit Kommern getrennt für das gesuchte Polynome:
$c_n,c_{n-1},...,c_0$

Erlaubt sind neben Ganz- und Gleitkommerzahlen auch Wurzelfunktion z.B.: sqrt(9)

In [1]:
import ipywidgets as widgets
from IPython.display import display
from polynom_interpolation.poly_inter import main 

def buttonClicked(_):
    main()

button = widgets.Button(description='Try Out', button_style='info')
display(button)
button.on_click(buttonClicked)

pygame 2.5.0 (SDL 2.28.0, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


Button(button_style='info', description='Try Out', style=ButtonStyle())

Durch das Lösen des Gleichungssystems, erhalten wir die Koeffizienten des Polynoms und können damit unser Polynom bauen. 

Die Polynom-Interpolation besitzt allerdings einige Nachteile:

1. Durch steigende Stützstellen (Grad des Polynoms) erreicht man nicht unbedingt eine höhere  Qualität. Es kann zu unschönen Ausschläge des Funktionsgraphen kommen.
2. Eine Veränderung von einem Punkt führt zur Änderung vom gesamten Funktionsgraphen.
3. Bewegungsrichtungen werden nicht beachtet. Die Bewegung eines abprallenden Balles kann so schlecht umgesetzt werden.

## Hermite Interpolation

Als Alternative zur Polynom-Interpolation bietet sich die Hermite-Interpolation an.

Bei der Hermite-Interpolation interpoliert man zwischen zwei Punkten in einem Intervall zwischen $0$ und $1$, bei mehrere Punkte werden einzelnen Kurvenstücken aneinander gehängt und ergeben somit eine ganze Kurve.

Zusätzlich zur Position $p_i$ wird noch die Steigung $m_i$ an jedem Punkt berücksichtigt, bei zwei Punkten mit jeweils einer Steigung hat man vier Gleichungen, man sucht also ein Polynom dritten Grades.

$f(t) = c_3 t^3 + c_2 t^2 + c_1 t + c_0 $

Die Steigung eines Punktes lässt sich durch die erste Ableitung ausdrücken:

$f'(t) = 3c_3 t^2 + 2c_2 t + c_1 $

daraus ergibt sich:

$f(0) = c_0$ , $f(1) = c_3+c_2+c_1+c_0$ , $f'(0) = c_1$ , $f'(1) = 3c_3+2c_2+c_1$

Das Gleichungssystem als Matrix: 


$$
\left(\begin{array}{cc} 
 1 & 0 & 0 & 0\\
0 & 1 & 0 & 0 \\
1 & 1 & 1 & 1 \\
0 & 1 & 2 & 3 \\
\end{array}\right)
\left(\begin{array}{cc} 
 c_0\\
c_1\\
c_2\\
c_3 
\end{array}\right)
=
\left(\begin{array}{cc} 
 f(0)\\
f'(1)\\
f(1)\\
f'(1)
\end{array}\right)
$$ 

Die lässt sich mit der Inversen lösen: 

$$
\left(\begin{array}{cc} 
 c_0\\
c_1\\
c_2\\
c_3 
\end{array}\right)
=
\left(\begin{array}{cc} 
 1 & 0 & 0 & 0\\
0 & 1 & 0 & 0 \\
-3 & -2 & 3 & -1 \\
2 & 1 & -2 & 1 \\
\end{array}\right)

\left(\begin{array}{cc} 
 f(0)\\
f'(1)\\
f(1)\\
f'(1)
\end{array}\right)
$$ 

Aus der Inversen lassen sich die Basisfunktionen ablesen:

$H_0^0 (t) = 2t^3 - 3t^2 +1$

$H_1^0 (t) = t^3-2t^2+t$

$H_0^1 (t) = -2t^3 + 3t^2$

$H_1^1 (t) = t^3-t^2$

Die Basisfunktionen können zur Interpolation genutzt werden, in dem man die Punkte bzw. deren Steigungen mit der entsprechenden Basisfunktion multipliziert und dann alles aufsummiert:

$p(t) = H_0^0 * p_0 + H_1^0 * m_0 + H_0^1 * p_1 + H_1^1 * m_1$

p(t) ist der interpolierte Punkt und zeichnet die Kurve.

Die Basisfunktionen dienen also als Gewichtungsfunktion für die Punkte bzw. Steigungen


Hier ein Demoprogramm, welches eine Hermite Interpolation visualisiert:

In [2]:
import ipywidgets as widgets
from IPython.display import display
from hermite.hermite import main

def buttonClicked(_):
    main()

button = widgets.Button(description='Try Out', button_style='info')
display(button)
button.on_click(buttonClicked)

Button(button_style='info', description='Try Out', style=ButtonStyle())

## Bézierkurve
### Allgemeines
Eine Bézierkurve ist eine parametrische Spline kurve, welche in der Computergrafik und im Design verbreitet ist. Diese wird durch einen Satz an Kontrollpunkten definiert, wobei der erste Punkt der Anfang der Kurve und der letzte Punkt das Ende der Kurve ist. Die restlichen Punkte haben einen Einfluss auf den Kurvenverlauf, wobei die Kurve, je nach Teil Stärker zu einem gewissen Punkt geneigt ist.
Bézierkurven tauschen in unterschiedlichen Graden, mit unterschiedlich vielen Kontrollpunkten auf, darunter die Lineare, quadratische und Kubische Bezierkurven.

Die lineare Bézierkurve besteht lediglich aus zwei punkten, welche mit einer Geraden verbunden sind.

Die Quadratische Bézierkurve wird durch drei Punkte definiert, einem Anfangs-, einem End- und einem Kontrollpunkt.

Die Kubische Bézierkurve hat 4 Punkte und sind die gebräuchlichsten, da diese durch eine erhöhte Kontrolle mehr Möglichkeiten bieten.

### Bernsteinpolynome
Die Gewichtung der Punkte wird bei einer Bezierkurve durch die Bernsteinpolynome ($B_0^3, B_1^3, B_2^3, B_3^3$) bestimmt, die Gewichtung ist von einer variable, oft t genannt abhängig, welche beeinflusst, zu welchem Zeitpunkt ein Kontrollpunkt wie stark gewichtet wird, dabei ist in der ersten Hälfte des Kurvenverlaufes die Kurve weitestgehend zu $B_1^3$ geneigt und in der zweiten Hälfte zu $B_2^3$.

Eine Kubische Bézierkurve b(t) sieht also mit den Bernstein Polynomen als Gewichtungsfunktion wie folgt aus:

$b(t) = B_0^3(t) * b0 + B_1^3(t) * b1 + B_2^3(t) * b2 + B_3^3(t) * b3$

### Casteljau Algorithmus
Durch die Bernsteinpolynome lässt sich der de Casteljau Algorithmus ableiten. Mit diesem kann eine Bézierkurve an der Stelle t dargestellt werden.

Der Algorithmus funktioniert wie folgt:
1. Gegeben sei eine Bézierkurve, definiert durch n+1 Kontrollpunkte $P_0, P_1, ..., P_n$.
2. Man berechnet die lineare Interpolation jedes benachbarten Paares von Kontrollpunkten für den Parameterwert t. Das ergibt n neue Punkte, die wir $b_0, b_1, ..., b_{(n-1)} $ nennen.
3. Man wiederholt den vorherigen Schritt für die neu berechneten Punkte, bis nur noch ein Punkt übrig bleibt. Dieser Punkt ist der Punkt auf der Bézierkurve für den Parameterwert t.

### Bézierkurve
Im Folgenden wurde ein Spiel implementiert, worin eine Bezierkurve gezeichnet wird, welche frei anhand der Kontrollpunkte bewegbar ist.
Wird das Spiel gestartet, so muss man mittels der Kontrollpunkte versuchen die Kurve durch die Hindernisse zu zeichnen.

In [1]:
import ipywidgets as widgets
from IPython.display import display
from bevier_curven.bezierGame import *

def buttonClicked(_):
    startBezierGame()

button = widgets.Button(description='Try Out', button_style='info')
display(button)
button.on_click(buttonClicked)

pygame 2.5.0 (SDL 2.28.0, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


Button(button_style='info', description='Try Out', style=ButtonStyle())

### Programm Bernstein Polynome
Hier wurde ein Programm geschrieben, welches die Bernsteinpolynome grafisch darstellt.

In [4]:
import ipywidgets as widgets
from IPython.display import display
from bevier_curven.bernsteinPolynomial import *

def buttonClicked(_):
    startPolynomGame()

button = widgets.Button(description='Try Out', button_style='info')
display(button)
button.on_click(buttonClicked)

Button(button_style='info', description='Try Out', style=ButtonStyle())